<a href="https://colab.research.google.com/github/kiran74-ds/zsl/blob/main/Embarssingly_Simple_ZSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://datasets.d2.mpi-inf.mpg.de/xian/xlsa17.zip

--2020-11-23 03:01:58--  http://datasets.d2.mpi-inf.mpg.de/xian/xlsa17.zip
Resolving datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)... 139.19.206.177
Connecting to datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)|139.19.206.177|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://datasets.d2.mpi-inf.mpg.de/xian/xlsa17.zip [following]
--2020-11-23 03:01:58--  https://datasets.d2.mpi-inf.mpg.de/xian/xlsa17.zip
Connecting to datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)|139.19.206.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 921803497 (879M) [application/zip]
Saving to: ‘xlsa17.zip’

xlsa17.zip          100%[===================>] 879.10M   103MB/s    in 23s     

2020-11-23 03:02:21 (38.7 MB/s) - ‘xlsa17.zip’ saved [921803497/921803497]



An embarrassingly simple approach to ZSL

The framework described with this approach is that of 2 layers which is built upon creating relationships between features, attributes and classes with the help of a linear model.

The first layer helps in defining the relationship between features and attributes with the help of weights in that layer.

The second layer deals with modelling the relationship between attributes and classes where the prescribed attribute signatures is fixed.

<img src="https://iq.opengenus.org/content/images/2020/01/Screenshot-from-2020-01-27-00-56-44.png">

In [ ]:
!unzip xlsa17.zip

In [3]:
import numpy as np
import os
import scipy.io
from sklearn.metrics import classification_report,confusion_matrix

Choose the one of the datasets from the options(APY, AWA1, AWA2, SUN, CUB).
I have selected AWA2(Animal with attributes data set) to implement model.

+ res101 contains Image Features extracted using resnet model.
+ att_splits contains splitting criteria for trian and test data along with class names information

In [97]:
dataset = 'AWA2'
res101 = scipy.io.loadmat('xlsa17/data/'+dataset+'/res101.mat')
att_splits = scipy.io.loadmat('xlsa17/data/'+dataset+'/att_splits.mat')

In [100]:
res101.keys()

dict_keys(['__header__', '__version__', '__globals__', 'features', 'image_files', 'labels'])

In [101]:
att_splits.keys()

dict_keys(['__header__', '__version__', '__globals__', 'allclasses_names', 'att', 'original_att', 'test_seen_loc', 'test_unseen_loc', 'train_loc', 'trainval_loc', 'val_loc'])

In [104]:
# extract the labels from att_split for all the data set points
labels = res101['labels']
labels_train = labels[np.squeeze(att_splits['train_loc']-1)]
labels_val = labels[np.squeeze(att_splits['val_loc']-1)]
labels_trainval = labels[np.squeeze(att_splits['trainval_loc']-1)]
labels_test = labels[np.squeeze(att_splits['test_unseen_loc']-1)]

In [105]:
# unique labels for all the data set points
train_labels_seen = np.unique(labels_train)
val_labels_unseen = np.unique(labels_val)
trainval_labels_seen = np.unique(labels_trainval)
test_labels_unseen = np.unique(labels_test)

In [107]:
# Indexing labels for all the data set points

i = 0
for labels in train_labels_seen:
    labels_train[labels_train == labels] = i    
    i = i+1
j = 0
for labels in val_labels_unseen:
    labels_val[labels_val == labels] = j
    j = j+1
k = 0
for labels in trainval_labels_seen:
    labels_trainval[labels_trainval == labels] = k
    k = k+1
l = 0
for labels in test_labels_unseen:
    labels_test[labels_test == labels] = l
    l = l+1

test_label_classes = []
for i in test_labels_unseen:
  test_label_classes.append(att_splits['allclasses_names'][i-1][0][0])

In [112]:
X_features = res101['features']
train_vec = X_features[:,np.squeeze(att_splits['train_loc']-1)]
val_vec = X_features[:,np.squeeze(att_splits['val_loc']-1)]
trainval_vec = X_features[:,np.squeeze(att_splits['trainval_loc']-1)]
test_vec = X_features[:,np.squeeze(att_splits['test_unseen_loc']-1)]


print("Features for train:", train_vec.shape)
print("Features for val:", val_vec.shape)
print("Features for trainval:", trainval_vec.shape)
print("Features for test:", test_vec.shape)

Features for train: (2048, 16187)
Features for val: (2048, 7340)
Features for trainval: (2048, 23527)
Features for test: (2048, 7913)


In [113]:
#Signature matrix
signature = att_splits['att']
train_sig = signature[:,(train_labels_seen)-1]
val_sig = signature[:,(val_labels_unseen)-1]
trainval_sig = signature[:,(trainval_labels_seen)-1]
test_sig = signature[:,(test_labels_unseen)-1]


In [82]:
#params for train and val set
m_train = labels_train.shape[0]
n_val = labels_val.shape[0]
z_train = len(train_labels_seen)
z1_val = len(val_labels_unseen)

#params for trainval and test set
m_trainval = labels_trainval.shape[0]
n_test = labels_test.shape[0]
z_trainval = len(trainval_labels_seen)
z1_test = len(test_labels_unseen)

In [83]:
#ground truth for train and val set
gt_train = 0*np.ones((m_train, z_train))
gt_train[np.arange(m_train), np.squeeze(labels_train)] = 1

#grountruth for trainval and test set
gt_trainval = 0*np.ones((m_trainval, z_trainval))
gt_trainval[np.arange(m_trainval), np.squeeze(labels_trainval)] = 1

In [84]:
#train set
d_train = train_vec.shape[0]
a_train = train_sig.shape[0]

#Weights
V = np.zeros((d_train,a_train))

In [85]:
#trainval set
d_trainval = trainval_vec.shape[0]
a_trainval = trainval_sig.shape[0]
W = np.zeros((d_trainval,a_trainval))

#Note: These hyper-parameters were found using the code snippet available below
gamm1 = 3
alph1 = 0

In [86]:
part_1_test = np.linalg.pinv(np.matmul(trainval_vec, trainval_vec.transpose()) + (10**alph1)*np.eye(d_trainval))
part_0_test = np.matmul(np.matmul(trainval_vec,gt_trainval),trainval_sig.transpose())
part_2_test = np.linalg.pinv(np.matmul(trainval_sig, trainval_sig.transpose()) + (10**gamm1)*np.eye(a_trainval))

W = np.matmul(np.matmul(part_1_test,part_0_test),part_2_test)


In [87]:
outputs_1 = np.matmul(np.matmul(test_vec.transpose(),W),test_sig)
preds_1 = np.array([np.argmax(output) for output in outputs_1])

In [88]:

cm = confusion_matrix(labels_test, preds_1)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
avg = sum(cm.diagonal())/len(test_labels_unseen)
print("The top 1% accuracy is:", avg*100)

The top 1% accuracy is: 45.73846664538193


In [89]:
print(classification_report(labels_test, preds_1, target_names=test_label_classes))

              precision    recall  f1-score   support

       horse       0.37      0.96      0.53      1645
  blue+whale       0.24      0.98      0.39       174
       sheep       0.51      0.06      0.10      1420
        seal       0.42      0.27      0.33       988
         bat       1.00      0.00      0.01       383
     giraffe       0.00      0.00      0.00      1202
         rat       0.35      0.97      0.51       310
      bobcat       0.82      0.87      0.85       630
      walrus       0.16      0.37      0.22       215
     dolphin       0.86      0.09      0.16       946

    accuracy                           0.39      7913
   macro avg       0.47      0.46      0.31      7913
weighted avg       0.46      0.39      0.29      7913



In [123]:
accu = 0.10
alph1 = 4
gamm1 = 1
for alpha in range(-3, 4):
    for gamma in range(-3,4):
        #One line solution
        part_1 = np.linalg.pinv(np.matmul(train_vec, train_vec.transpose()) + (10**alpha)*np.eye(d_train))
        part_0 = np.matmul(np.matmul(train_vec,gt_train),train_sig.transpose())
        part_2 = np.linalg.pinv(np.matmul(train_sig, train_sig.transpose()) + (10**gamma)*np.eye(a_train))

        V = np.matmul(np.matmul(part_1,part_0),part_2)
        #print(V)

        #predictions
        outputs = np.matmul(np.matmul(val_vec.transpose(),V),val_sig)
        preds = np.array([np.argmax(output) for output in outputs])

        print(accuracy_score(labels_val,preds))
        cm = confusion_matrix(labels_val, preds)
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        avg = sum(cm.diagonal())/len(val_labels_unseen)
        #print("Avg:", avg, alpha, gamma)

        if avg > accu:
            accu = avg
            alph1 = alpha
            gamm1 = gamma
print(alph1, gamm1)

NameError: ignored